In [8]:
f_func <- "../functions.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_func, '--to=script', '--stdout'), stdout = TRUE)))
f_path <- "../dirs.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_path, '--to=script', '--stdout'), stdout = TRUE)))

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

[[11]]
[1] TRUE

[[12]]
[1] TRUE

[[13]]
[1] TRUE

[[14]]
[1] TRUE

[[15]]
[1] TRUE

[[16]]
[1] TRUE

[[17]]
[1] TRUE

[[18]]
[1] TRUE

[[19]]
[1] TRUE

In [9]:
Asp_species_perDB <- list()
for (db in dbs){
    if (db == "EPRS"){
        in_db_insp <- paste0(DB_DIR, db, "_miss2gen_inspect.txt")}
    else {in_db_insp <- paste0(DB_DIR, db, "_inspect.txt")}
    db_insp <- read.csv(file = in_db_insp, header=FALSE, sep = "\t", stringsAsFactors=FALSE)
    db_insp$V6 <-  str_trim(db_insp$V6, side = "left")
    Asp_species_perDB[[db]] <- db_insp  %>%   
        filter(grepl(V6, pattern = "^Aspergillus "))  %>% 
        filter(!grepl(V6, pattern = "virus"))  %>% 
        filter(V4 == "S") %>% pull("V6")
}

In [10]:
if (exists("kmers_species_Asp")){rm("kmers_species_Asp")}
for (db in dbs){
    if (db == "EPRS"){
        in_db_insp <- paste0(DB_DIR, db, "_miss2gen_inspect.txt")}
    else {in_db_insp <- paste0(DB_DIR, db, "_inspect.txt")}
    db_insp <- read.csv(file = in_db_insp, header=FALSE, sep = "\t", stringsAsFactors=FALSE)
    db_insp$V6 <-  str_trim(db_insp$V6, side = "left")

    temp_kmers_Asp <- db_insp  %>% mutate("db" = db) %>% 
        filter(grepl(V6, pattern = "^Aspergillus "))  %>% 
        filter(!grepl(V6, pattern = "virus"))  %>% 
        filter(V4 == "S") %>% pull("V2", "V6") %>% 
        as.data.frame()
    if (db == "EPRS_miss2gen"){temp_kmers_Asp$db <- "EPRS"} else {temp_kmers_Asp$db <- db}
    colnames(temp_kmers_Asp) <- c("kmer_count", "db")
    if (!exists("kmers_species_Asp")){ 
        kmers_species_Asp <- temp_kmers_Asp
    } else {
        kmers_species_Asp <- rbind(kmers_species_Asp, temp_kmers_Asp)
    }
}

In [11]:
oi <- c("Archaea", "Bacteria", "Viruses", "Eukaryota", "Homo sapiens", "Fungi", "Aspergillus")
mt <- matrix(ncol = length(oi), nrow = length(dbs))
colnames(mt) <- oi
rownames(mt) <- dbs
rownames(mt)[which(rownames(mt) == "EPRS_miss2gen")] <- "EPRS"

In [12]:
mt_nr_species <- matrix(ncol = 2, nrow = length(dbs))
colnames(mt_nr_species) <- c("species", "subgenera")
rownames(mt_nr_species) <- dbs

for (db in rownames(mt_nr_species)){
    if (db == "EPRS"){
        in_db_insp <- paste0(DB_DIR, db, "_miss2gen_inspect.txt")}
    else {in_db_insp <- paste0(DB_DIR, db, "_inspect.txt")}
    
    db_insp <- read.csv(file = in_db_insp, header=FALSE, sep = "\t", stringsAsFactors=FALSE)
    db_insp$V6 <-  str_trim(db_insp$V6, side = "left")

    mt_nr_species[db,"species"] <- db_insp  %>%  
        filter(grepl(V6, pattern = "^Aspergillus "))  %>% 
        filter(!grepl(V6, pattern = "virus"))  %>% 
        filter(V4 == "S")  %>% nrow()
    
    mt_nr_species[db,"subgenera"] <- db_insp  %>%  
        filter(grepl(V6, pattern = "^Aspergillus "))  %>% 
        filter(!grepl(V6, pattern = "virus"))  %>% 
        filter(V4 == "G1")  %>% nrow()
}

In [13]:
mt_inspect <- mt 
for (db in rownames(mt_inspect)){
    if (db == "EPRS"){
        in_db_insp <- paste0(DB_DIR, db, "_miss2gen_inspect.txt")}
    else {in_db_insp <- paste0(DB_DIR, db, "_inspect.txt")}
    db_insp <- read.csv(file = in_db_insp, header=FALSE, sep = "\t", stringsAsFactors=FALSE)
    db_insp$V6 <-  str_trim(db_insp$V6, side = "left")

    for (i in colnames(mt_inspect)){   
        mt_inspect[db,i] <- db_insp[db_insp$V6 == i,"V2"]
    }
}

In [14]:
saveRDS(mt_nr_species,    paste0(INT_database_stats, "mt_nr_species.Rdata"))
saveRDS(mt_inspect,       paste0(INT_database_stats, "mt_inspect.Rdata"))